In [41]:
# Mount Google Drive so we can access data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
# Install idx2numpy package for extracting data
!pip install idx2numpy

In [43]:
# Import packages
import os
import json
import gzip
import torch
import torchvision
import numpy as np 
import pandas as pd

import idx2numpy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms.functional as transforms
import matplotlib.pyplot as plt

In [44]:
def load_one_dataset(path):
    '''
    Convenience function to load a single dataset
    '''
    f = gzip.open(path, 'rb')
    data = torch.from_numpy(idx2numpy.convert_from_file(f))
    f.close()
    
    return(data)


def load_all_datasets(train_imgs, train_labs, test_imgs, test_labs, batch_size):
    '''
    Load training as well as test images here
    '''
    train_images = load_one_dataset(train_imgs).type(torch.float32)
    train_labels = load_one_dataset(train_labs).type(torch.long)
    train = list(zip(train_images, train_labels))

    test_images = load_one_dataset(test_imgs).type(torch.float32)
    test_labels = load_one_dataset(test_labs).type(torch.long)
    test = list(zip(test_images, test_labels))
    
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=2)
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False, num_workers=2)
    
    return(train_loader, test_loader)

In [45]:
def add_noise(img, quadrants):
  '''
  Randomly remove 1 or 2 quadrants
  from the input image.
  '''
  # Get the number of quadrants to erase
  n_quads_to_erase = np.random.choice([1, 2])

  # Get which quadrants to erase
  quads_to_erase = np.random.choice([1, 2, 3, 4], size = n_quads_to_erase)

  # Create a copy of the image
  noisy_img = img.clone()

  # Now erase the quadrants
  for quad in quads_to_erase:
    noisy_img = transforms.erase(noisy_img, *quadrants[quad])
  
  # Return statement
  return(noisy_img)

In [46]:
def plot_image(img):
  '''
  Take an image stored as a Torch
  tensor and display it in the notebook
  '''
  # Display the img
  plt.imshow(img.numpy(), cmap= 'gray')

In [47]:
def test_noise():
  '''
  Test the noise function
  '''
  # Just for testing out noise function
  data_dir = '/content/drive/MyDrive/data'

  # Set paths
  paths = {
        'train_imgs': os.path.join(data_dir, 'train-images-idx3-ubyte.gz'),
        'train_labs': os.path.join(data_dir, 'train-labels-idx1-ubyte.gz'),
        'test_imgs': os.path.join(data_dir,'t10k-images-idx3-ubyte.gz'),
        'test_labs': os.path.join(data_dir,'t10k-labels-idx1-ubyte.gz')
  }

  # Load datasets
  train_loader, test_loader = load_all_datasets(**paths, batch_size = 32)

  # Get the next batch from the train loader
  images, labels = iter(train_loader).next()

  # Store the quadrant definitions: move this into training loop later
  quadrants = {
      
      1: [0, 0, 14, 14, 255], 
      2: [0, 14, 14, 14, 255],
      3: [14, 0, 14, 14, 255],
      4: [14, 14, 14, 14, 255],
  }

  # Take the first image in the batch
  img = images[0]

  # Get noisy image
  noisy_img = add_noise(img, quadrants)

  # Return statement
  return(img, noisy_img)

In [48]:
class DenoisingEncoder(nn.Module):
  
  def __init__(self):
    
    super(DenoisingEncoder,self).__init__()
    
    self.encoder=nn.Sequential(
                  nn.Linear(28*28,256),
                  nn.ReLU(True),
                  nn.Linear(256,128),
                  nn.ReLU(True),
                  nn.Linear(128,64),
                  nn.ReLU(True)
        
                  )
    
    self.decoder=nn.Sequential(
                  nn.Linear(64,128),
                  nn.ReLU(True),
                  nn.Linear(128,256),
                  nn.ReLU(True),
                  nn.Linear(256,28*28),
                  nn.Sigmoid(),
                  )
    
 
  def forward(self,x):
    
    x = x.view(-1, 28*28)
    x=self.encoder(x)
    x=self.decoder(x)
    
    return x

In [49]:
def train(epochs=10, batch_size=32, data_dir = '/content/drive/MyDrive/data'):
    '''
    This is the main training loop
    '''
    # Set device
    if torch.cuda.is_available():
      device = torch.device("cuda")
    else:
      device = torch.device("cpu")
    
    # Set paths to datasets
    paths = {
        'train_imgs': os.path.join(data_dir, 'train-images-idx3-ubyte.gz'),
        'train_labs': os.path.join(data_dir, 'train-labels-idx1-ubyte.gz'),
        'test_imgs': os.path.join(data_dir,'t10k-images-idx3-ubyte.gz'),
        'test_labs': os.path.join(data_dir,'t10k-labels-idx1-ubyte.gz')
    }

    # Store the quadrant definitions: move this into training loop later
    quadrants = {
      
      1: [0, 0, 14, 14, 255], 
      2: [0, 14, 14, 14, 255],
      3: [14, 0, 14, 14, 255],
      4: [14, 14, 14, 14, 255],
    }

    # Load datasets
    train_loader, test_loader = load_all_datasets(**paths, batch_size = batch_size)
    
    # Set parameters
    net = DenoisingEncoder()
    
    # Send net object to device memory
    net.to(device)
    
    # We use the cross-entropy loss
    criterion = nn.MSELoss()

    # We use mini-batch stochastic gradient descent with momentum
    optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.09, 
                                            weight_decay=0)

    # Store results here
    results = {
      'train_loss': [], 
      'val_loss': [], 
      }

    # Loop over the dataset multiple times
    for epoch in range(epochs):  
        
        # Initialize running loss
        running_loss = 0.0
        running_accuracy = 0.0

        # Initialize the validation running loss
        val_running_loss = 0.0
        val_running_accuracy = 0.0
        
        # Iterate through data now
        for i, data in enumerate(train_loader):
            
            # Get the inputs: data is a list of [inputs, labels]
            clean_images, _ = data
            
            # Initialize container for noisy images
            noisy_images = []
            flat_clean_imgs = []

            # Now get noisy images
            for img in clean_images: 
              noisy_images.append(add_noise(img, quadrants))
              flat_clean_imgs.append(img.view(-1, 28*28))
            
            # Convert noisy image list to Torch tensor
            noisy_images = torch.stack(noisy_images, dim =0)
            flat_clean_imgs = torch.stack(flat_clean_imgs, dim=0)

            # Send the inputs and labels to the memory of the device
            noisy_images, clean_images = noisy_images.to(device), clean_images.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward
            pred_images = net(noisy_images)
            
            # Calculate loss
            loss = criterion(pred_images, flat_clean_imgs)

            # Backward
            loss.backward()
            
            # Optimize
            optimizer.step()

            # Add to running loss
            running_loss += loss.item()
        
        # Loop through the validation data
        for j, data in enumerate(test_loader):
          
          # No need to calculate gradients for validation set
          with torch.no_grad():

              # Get the inputs: data is a list of [inputs, labels]
              clean_images, _ = data
            
              # Initialize container for noisy images
              noisy_images = []
              flat_clean_imgs = []

              # Now get noisy images
              # Flatten clean images for loss calculation
              for img in clean_images: 
                noisy_images.append(add_noise(img, quadrants))
                flat_clean_imgs.append(img.view(-1, 28*28))
            
              # Convert noisy image list to Torch tensor
              noisy_images = torch.stack(noisy_images, dim=0)
              flat_clean_imgs = torch.stack(flat_clean_imgs, dim=0)

              # Send the inputs and labels to the memory of the device
              noisy_images, clean_images = noisy_images.to(device), clean_images.to(device)

              # Send the data item through the network to get output
              val_pred_images = net(noisy_images)

              # Compute the loss
              val_loss = criterion(val_pred_images, flat_clean_imgs)

              # Add to running loss
              val_running_loss += val_loss.item()
        
        # Rescale the training and validation perfomance metrics
        running_loss = running_loss/len(train_loader)
        
        # Rescale the validation loss
        val_running_loss = val_running_loss/len(test_loader)
        
        # Append to the results tracker
        results['train_loss'].append(np.float(running_loss))
        results['val_loss'].append(np.float(val_running_loss))

        # Make print message format string
        msg = "{}, Epoch:{}, Loss:{}" "\n"

        # Print performance
        print(msg.format("Training", epoch, running_loss))
        print(msg.format("Validation", epoch, val_running_loss))
        
    # Print message
    print('Done training...')
    
    # Return statement
    return(results)

In [50]:
train()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([32, 1, 784])) that is different to the input size (torch.Size([32, 784])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: ignored